In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver # módulo que contiene implementaciones de controladores de navegadores web
from webdriver_manager.chrome import ChromeDriverManager # controlador de Chrome  
from selenium.webdriver.support import expected_conditions as EC # método para escribir códigos que esperan hasta que ciertas condiciones sean cumplidas
from selenium.webdriver.support.ui import WebDriverWait # método para escribir códigos que utilizan esperas implícitas o explícitas
from selenium.webdriver.common.by import By # método para localizar elementos por sus atributos HTML
from selenium.webdriver import ActionChains # módulo para implementar interacciones con el navegador from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [2]:
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
#d = webdriver.Chrome('your-chrome-driver-path',chrome_options=chrome_options)

In [3]:
# Inicializa la página web en el navegador Chrome
driver = webdriver.Chrome(ChromeDriverManager().install(),chrome_options=chrome_options)
url = 'https://www.despegar.com.co/'
driver.get(url) # aca le pido al webdriver que me traiga la pagina que le solicite en url

[WDM] - Current google-chrome version is 97.0.4692
[WDM] - Get LATEST driver version for 97.0.4692


[WDM] - Get LATEST driver version for 97.0.4692
[WDM] - Trying to download new driver from http://chromedriver.storage.googleapis.com/97.0.4692.71/chromedriver_linux64.zip
[WDM] - Driver has been saved in cache [/home/genesis/.wdm/drivers/chromedriver/linux64/97.0.4692.71]


In [4]:
elemento_oferta = '.shifu-3-header .header-corners-container .header-product-item .shifu-3-button-circle.OFFERS'
delay = 10
try:
    element_present = EC.presence_of_element_located((By.CSS_SELECTOR, elemento_oferta))
    WebDriverWait(driver, delay).until(element_present)
except TimeoutError:
    print('Timed out waiting for page to load')
print(driver.current_url)


https://www.despegar.com.co/


In [5]:
element1 = driver.find_element_by_css_selector(elemento_oferta)
ActionChains(driver).click(element1).perform()
print(driver.current_url)

https://www.despegar.com.co/ofertas-de-viajes


In [6]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
driver.quit()


In [7]:
etiqueta_promociones = soup.select('slider-item')  # selecciona todos los elementos que tienen la clase slider-item



In [32]:
def chequear_nulo(etiqueta):
    if etiqueta is not None:
        return etiqueta.text

def chequear_imagen(etiqueta):
    if etiqueta is not None:
        return etiqueta['src']

for etiqueta in etiqueta_promociones:
    titulo = etiqueta.select_one('.offer-card-title.small-title ')
    imagen = etiqueta.select_one('.offer-card-image .offer-card-image-main')
    categoria = etiqueta.select_one('.offer-card-product')
    precio_oferta = etiqueta.select_one('.offer-card-pricebox-price-amount ')
    precio_normal = etiqueta.select_one('.offer-card-pricebox-price .offer-card-pricebox-price-old')
    print(chequear_nulo(titulo), chequear_imagen(imagen), chequear_nulo(categoria), chequear_nulo(precio_oferta), chequear_nulo(precio_normal))
    print('\n')




Paquetes a Cartagena de Indias //media.staticontent.com/media/pictures/6b4d7593-f154-4c1c-a0b8-5aefdcca9b23/300x200 Paquete 738.644  $1.461.227 


Paquetes a Cali //media.staticontent.com/media/pictures/cf17bae5-5bbf-4b67-a1d4-c79fab7f3ea0/300x200 Paquete 337.614  $456.699 


Paquetes a San Andrés //media.staticontent.com/media/pictures/5cc13b62-c0dc-4143-b3e0-a895752c280e/300x200 Paquete 976.737  $1.620.956 


Paquetes a Cancún //media.staticontent.com/media/pictures/3f50e52f-a644-4d87-aabf-26d65078c5ec/300x200 Paquete 2.371.837  $2.696.131 


Paquetes a Ciudad de México //media.staticontent.com/media/pictures/396b2633-76a8-4709-9a5f-f157beaa0dfc/300x200 Paquete 1.197.824  $1.788.435 


Paquetes a Nueva York //media.staticontent.com/media/pictures/8ddc824a-122f-44c7-b906-ec91cbfb5be5/300x200 Paquete 1.925.338  $2.065.457 


Paquetes a Panamá //media.staticontent.com/media/pictures/47408bb2-90b3-41f7-aa07-7514c25f7480/300x200 Paquete 1.124.728  $1.381.099 


Paquetes a Buenos Aires //m

In [28]:
total_promociones = {'categoria':[],'titulo':[],'precio_oferta':[],'precio_normal':[]}
for promociones in etiqueta_promociones:
    total_promociones['categoria'].append(chequear_nulo(promociones.select_one('.offer-card-product')))
    total_promociones['titulo'].append(chequear_nulo(promociones.select_one('.offer-card-title.small-title')))
    total_promociones['precio_oferta'].append(chequear_nulo(promociones.select_one('.offer-card-pricebox-price-amount')))
    total_promociones['precio_normal'].append(chequear_nulo(promociones.select_one('.offer-card-pricebox-price .offer-card-pricebox-price-old')))

print(total_promociones)
print('\n')	

{'categoria': ['Paquete', 'Paquete', 'Paquete', 'Paquete', 'Paquete', 'Paquete', 'Paquete', 'Paquete', 'Vuelo', 'Vuelo', 'Vuelo', 'Vuelo', 'Vuelo', 'Vuelo', 'Vuelo', 'Vuelo', 'Vuelo', 'Vuelo', 'Vuelo', 'Vuelo', 'Alojamiento', 'Actividad', 'Alojamiento', 'Vuelo', 'Vuelo', 'Paquete', 'Alojamiento', 'Auto', 'Auto', 'Vuelo', 'Alojamiento', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None], 'titulo': ['Paquetes a Cartagena de Indias', 'Paquetes a Cali', 'Paquetes a San Andrés', 'Paquetes a Cancún', 'Paquetes a Ciudad de México', 'Paquetes a Nueva York', 'Paquetes a Panamá', 'Paquetes a Buenos Aires', 'Vuelo a Cancún', 'Vuelo a Miami', 'Vuelo a Guarullos', 'Vuelo a Madrid', 'Vuelo a Cartagena de Indias', 'Vuelo a Santa Marta', 'Vuelo a San Andrés', 'Vuelo a Medellín', 'Vuelos a Medellín', 'Vuelos a Cartagena de Indias', 'Vuelos a Santa Marta', 'Vuelos a Cali', 'UGO Hotel', 'Entradas a Parques Temáticos Disney',